In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import os
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import h5py
import math
import pickle
from datetime import datetime
from numba import cuda 

%matplotlib inline

In [2]:
import sys  
sys.path.insert(0, '..')

In [3]:
from tiling.read_tiles import TissueDataset, load_color_normalization_values
from tiling.preprocessing.processing import split_whole_slide
from tiling.preprocessing.datamodel import SlideManager, Slide

In [4]:
# Code snippet nvidia gpu
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
HDF5_FOLDER = '/media/nico/data/fourthbrain/project/training_CAMELYON16'
green_layer_only = False
color_normalization_file="/home/nico/Documents/FourthBrainBreastCancer/tiling/CAMELYON16_color_normalization.json"

In [6]:
# FEATURE MAP CONSTRUCTION

In [7]:
# FIRST GET TILES

In [8]:
CAM_BASE_DIR = '/media/nico/data/fourthbrain/project/'
CAM16_DIR = CAM_BASE_DIR + 'CAMELYON16/'
color_normalization_file="/home/nico/Documents/FourthBrainBreastCancer/tiling/CAMELYON16_color_normalization.json"

In [9]:
mgr = SlideManager(cam16_dir=CAM16_DIR)

In [10]:
i = 0
level = 3
tile_size=256
verbose = True

In [11]:
slide = mgr.test_slides[i]

In [20]:
iter_whole_slide = split_whole_slide(slide, level=level, tile_size=tile_size, color_normalization_file=color_normalization_file, verbose=False) 

In [21]:
model = keras.models.load_model('/media/nico/data/fourthbrain/project/model_testing/model_inceptionv3.hdf5')

In [24]:
tiles, labels, coords, slide_locations = list(), list(), list(), list()

y_preds, layer_outputs = list(), list()
for tile, label, coord, slide_location in iter_whole_slide:
#     tiles.append(tile)
    print(tile.shape)
#     labels.append(label)
#     coords.append(coord)
#     slide_locations.append(slide_location)
    y_pred = model.predict(tile)
    
    print(y_pred)
#     layer_outputs.append(model.layers[-1].output)


(256, 256, 3)


ValueError: in user code:

    /home/nico/miniconda3/envs/nightly/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1510 predict_function  *
        return step_function(self, iterator)
    /home/nico/miniconda3/envs/nightly/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1500 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/nico/miniconda3/envs/nightly/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1263 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/nico/miniconda3/envs/nightly/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2735 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/nico/miniconda3/envs/nightly/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3424 _call_for_each_replica
        return fn(*args, **kwargs)
    

In [15]:
# Model layer

layer_output = model.layers[-1].output



# feature_model = Model(inputs=model_googlenet.input, outputs=layer_output)
# feature_map_total = []
# for x in range(0,len(X_train)):
#   feature_map = feature_model.predict(X_train[x].reshape(1,256,256,3)) 
#   feature_map_total.append(feature_map)

In [16]:
layer_output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'predictions')>

In [17]:
from keras.models import Model

In [18]:
# Construct feature maps and reshape them into 256*256*3 for UNet
model = Model(inputs=feat_model.inputs, outputs=feat_model.layers[83].output)


layer_outputs = [layer.output for layer in model.layers]
feature_model = Model(inputs=model.input, outputs=layer_outputs)
feature_map_total = list()

NameError: name 'feat_model' is not defined

In [ ]:
# Generate batch 
iter_train = train_data.generator(num_neg=300, num_pos=300, data_augm=False,
                  color_normalization_file=color_normalization_file,
                  green_layer_only=False)
# Use the generator
i = 0
X_train, y_train = list(), list()

In [ ]:
for x, y in iter_train:
    X_train.append(x)
    y_train.append(y)
    i+=1
    if i > 0:
        break

In [ ]:
X_train = X_train[0]
y_train = y_train[0]

In [ ]:
for x in range(0,len(X_train)):
    feature_map = feature_model.predict(X_train[x].reshape(1,256,256,3)) 
    feature_map_total.append(feature_map)